<a href="https://colab.research.google.com/github/algo21-221040114/PytorchTest/blob/main/Model_Transformer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sweetviz

In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import sweetviz as sv
from pandas_profiling import ProfileReport

In [1]:
import pandas as pd
import numpy as np

import math
import torch
import torch.nn as nn

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/sample_data/000001.SS.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-16,3107.669922,3177.790039,3023.300049,3170.709961,3170.709961,444700
2022-03-17,3215.010010,3260.169922,3202.929932,3215.040039,3215.040039,420800
2022-03-18,3207.149902,3260.780029,3197.360107,3251.070068,3251.070068,373000
2022-03-21,3255.620117,3267.500000,3223.389893,3253.689941,3253.689941,354200
2022-03-22,3249.540039,3279.110107,3239.530029,3259.860107,3259.860107,361200


In [ ]:
# Plot
fig, ax = plt.subplots()
ax.plot(df.index, df['High'], color='y', label='high')
ax.plot(df.index, df['Close'], color='b', label='close')
ax.set_title('000001.SS')
ax.legend()
plt.show()

In [ ]:
# EDA 
def eda_sweetviz(df, report_name, target=None):
    # target certain data
    sweet_report = sv.analyze(df, target_feat=target)
    # in notebook, download the file and open in the browser
    sweet_report.show_html(f'{report_name}.html')

def eda_sweetviz_compare(df1, df2, report_name, feature, target):
    # compare two datasets
    feature_config = sv.FeatureConfig(force_text=feature, force_cat=feature)
    sweet_report = sv.compare(df1, df2, feat_cfg=feature_config, target_feat=target)
    sweet_report.show_html(f'{report_name}_compare.html')


eda_sweetviz(df, 'Daily Data of 000001.SS')

def eda_pandas_profilling(df, report_name):
    # directly show in notebook
    profile = ProfileReport(df, title=report_name, explorative=True)
    return profile

eda_pandas_profilling(df, '000001.SS Report')

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)*(-math.log(10000.0)/d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
# Normalization 


In [ ]:
# Data Preprocess
cl = df['Close']
# features = np.array((df['Open'], df['High'], df['Low'], df['Close'])).T
features = np.array((df['Close'])).T
length = features.shape[0] # total time length
src = []
tgt = []
tgt_y = []
pre_len = 5

for i in range(length-pre_len-2):
  s = features[i: i+pre_len]
  src.append(s)

  t = features[i+pre_len+1]
  tgt.append(t)

  t_y = features[i+pre_len+1] # close price
  tgt_y.append(t_y)

train_src = src[:200]
test_src = src[200:]

train_tgt = tgt[:200]
test_tgt = tgt[200:]

train_tgt_y = tgt_y[:200]
test_tgt_y = tgt_y[200:]

print(len(train_tgt_y))

In [71]:
def shuffle_data(feature, target, target_y):
    """
    :param feature: encoder输入特征, list
    :param target: decoder输入特征
    :param target_y: ground truth
    :param shuffle: 是否打乱顺序
    :return:
    """
    # 输入all_datas的每一项必须是numpy数组
    feature = [np.array(x) for x in feature]
    target = [np.array(x) for x in target]
    target_y = [np.array(x) for x in target_y]

    # 获取样本大小
    data_size = len(feature)

    # 随机生成打乱的索引
    p = np.random.permutation(data_size)
    # 重新组织数据
    feature = torch.Tensor(np.array(feature)[p]).unsqueeze(-1)
    target =  torch.Tensor(np.array(target)[p]).unsqueeze(1).unsqueeze(-1)
    target_y =  torch.Tensor(np.array(target_y)[p])

    return feature, target, target_y

In [72]:
src, tgt, tgt_y = shuffle_data(train_src, train_tgt, train_tgt_y)
print(src.size())
print(tgt.size())
print(tgt_y.size())

torch.Size([200, 5, 1])
torch.Size([200, 1, 1])
torch.Size([200])


In [73]:
# 策略：使用前十个交易日的close信息预测当日close
class QuantModel(nn.Module):

  def __init__(self, n_feature):
      super(QuantModel, self).__init__()

      self.transformer = nn.Transformer(d_model = n_feature,
                                        nhead=1,
                                        num_encoder_layers=12,
                                        num_decoder_layers=12,
                                        dim_feedforward=128,
                                        dropout=0.1,
                                        activation='relu',
                                        custom_encoder=None,
                                        custom_decoder=None,
                                        batch_first = True)
      self.out_linear = nn.Linear(n_feature, 1)

  def forward(self, src, tgt):

      tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size()[1])
      
      out = self.transformer(src, tgt, tgt_mask=tgt_mask)

      return out

model = QuantModel(n_feature=1)

In [74]:
criteria = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-2)

In [75]:
# Train model
data_size = 200
batch_size = 16
total_loss = 0
epochs = 100
batch_count = data_size//batch_size

for epoch in range(1, epochs + 1):

    total_loss = 0
    optimizer.zero_grad()
    src, tgt, tgt_y = shuffle_data(train_src, train_tgt, train_tgt_y)

    for batch in range(batch_count+1):
      start = batch*batch_size
      if start >= data_size:
        break
      else:
        end = min((batch+1)*batch_size, data_size)
          
        src1 = src[start:end]
        tgt1 = tgt[start:end]
        tgt_y1 = tgt_y[start:end]
        out = model(src1, tgt1)
        out = model.out_linear(out)
        loss = criteria(out.contiguous().view(-1, out.size(-1)), 
                        tgt_y1.contiguous().view(-1, tgt_y1.size(-1)))

        loss.backward()
        optimizer.step()
        total_loss += loss

    # 每10次打印一下loss
    if epoch != 0 and epoch % 5 == 0:
        print("Epoch {}, total_loss: {}".format(epoch, total_loss))
        total_loss = 0

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 5, total_loss: 129489432.0
Epoch 10, total_loss: 124956840.0
Epoch 15, total_loss: 116929456.0
Epoch 20, total_loss: 106027416.0
Epoch 25, total_loss: 93013232.0
Epoch 30, total_loss: 78551928.0
Epoch 35, total_loss: 63991448.0
Epoch 40, total_loss: 50158924.0
Epoch 45, total_loss: 37412156.0
Epoch 50, total_loss: 26723344.0
Epoch 55, total_loss: 18010996.0
Epoch 60, total_loss: 11476670.0
Epoch 65, total_loss: 6913184.0
Epoch 70, total_loss: 3883216.0
Epoch 75, total_loss: 2094263.375
Epoch 80, total_loss: 1100077.875
Epoch 85, total_loss: 590417.6875
Epoch 90, total_loss: 344843.25
Epoch 95, total_loss: 222370.3125
Epoch 100, total_loss: 181825.546875


In [76]:
# Test model
model = model.eval()

In [78]:
test_src = torch.Tensor(test_src).unsqueeze(-1)
test_tgt = torch.Tensor(test_tgt).unsqueeze(-1).unsqueeze(-1)

In [83]:
test_len = test_src.size()[0]
print(test_len)

37


In [80]:
test_src0 = test_src[0].unsqueeze(0)
test_tgt0 = test_tgt[0].unsqueeze(0)

In [ ]:
prediction = []
for i in range(test_len):
  if i != 0:
    copy = torch.clone(test_src0)
    test_src0[0, :-1, :] = copy[0, 1:, :]
    test_src0[0, -1, :] = test_tgt0[0, -1, :]
    test_tgt0[0, -1, :] = prediction[-1]
  output = model(test_src0, test_tgt0)
  output = model.out_linear(output) 
  prediction.append(float(output[0])) 


fig, ax = plt.subplots()
plt.xticks(rotation=60)
ax.plot(df.index[-test_len:], prediction, color='y', label='pred close')
ax.plot(df.index[-test_len:], df['Close'][-test_len:], color='b', label='close')
ax.set_title('000001.SS')
ax.legend()
plt.show()